In [59]:
import pandas as pd
import numpy as np

from pandas import json_normalize

from typing import Union
from io import StringIO

import yaml

In [60]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path, encoding='utf-8'), Loader=yaml.FullLoader)

preproc = config['preprocessing']
# evaluate = config['evaluate']

In [61]:
df = pd.read_csv(preproc['raw_train_data_path'], sep=';')
df[:5]

,Inn,balance_data,god,lik_m
0,1635011279,"{""КНД"": ""0710099"", ""ОКЕИ"": 384, ""СвНП"": {""НПЮЛ...",2022,2099-12-31
1,1628007031,"{""КНД"": ""0710099"", ""ОКЕИ"": 384, ""СвНП"": {""НПЮЛ...",2022,2099-12-31
2,1632007852,"{""КНД"": ""0710099"", ""ОКЕИ"": 384, ""СвНП"": {""НПЮЛ...",2022,2099-12-31
3,1639049890,"{""КНД"": ""0710099"", ""ОКЕИ"": 384, ""СвНП"": {""НПЮЛ...",2022,2099-12-31
4,1677000712,"{""КНД"": ""0710099"", ""ОКЕИ"": 384, ""СвНП"": {""НПЮЛ...",2022,2099-12-31


# Распаковка

In [62]:
df['balance_data'][0]

'{"КНД": "0710099", "ОКЕИ": 384, "СвНП": {"НПЮЛ": {"КПП": "163501001", "АдрМН": "422116, Респ.Татарстан, Кукморский р-н, с.Каркаусь, ул.Ленина, д.104", "ИННЮЛ": "1635011279", "НаимОрг": "Общество с ограниченной ответственностью \\"УНЫШ\\""}, "ОКПО": "47085707", "ОКФС": "16", "ОКОПФ": "12300", "ОКВЭД2": "01.50"}, "Баланс": {"ОКУД": "0710001", "Актив": {"ОбА": {"ДебЗад": {"СумОтч": 62430, "СумПрдщ": 2678}, "Запасы": {"СумОтч": 111071, "СумПрдщ": 100760, "СумПрдшв": 86693}, "СумОтч": 173703, "ПрочОбА": {}, "СумПрдщ": 103963, "ФинВлож": {}, "ДенежнСр": {"СумОтч": 202, "СумПрдщ": 525}, "СумПрдшв": 86693, "НДСПриобрЦен": {}}, "ВнеОбА": {"ОснСр": {"СумОтч": 60449, "СумПрдщ": 34200, "СумПрдшв": 33757}, "СумОтч": 82252, "СумПрдщ": 49487, "ФинВлож": {}, "ВлМатЦен": {}, "НематАкт": {}, "СумПрдшв": 45095, "ОтлНалАкт": {}, "РезИсслед": {}, "ПрочВнеОбА": {"СумОтч": 21803, "СумПрдщ": 15287, "СумПрдшв": 11338}}, "СумОтч": 255955, "СумПрдщ": 153450, "СумПрдшв": 131788, "МатПоискАкт": {}, "НеМатПоискАкт

In [63]:
def safe_json_loads(x: str) -> Union[dict, None]:
    """
    Преобразование json-данных в соответствующий объект python
    :param x: путь до файла со словарем
    :return: преобразованный объект python
    :return: None
    """
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return None

In [64]:
df[preproc['unpack_col_name']] = df[preproc['unpack_col_name']].apply(
    safe_json_loads)
df[preproc['unpack_col_name']][0]

{'КНД': '0710099',
 'ОКЕИ': 384,
 'СвНП': {'НПЮЛ': {'КПП': '163501001',
   'АдрМН': '422116, Респ.Татарстан, Кукморский р-н, с.Каркаусь, ул.Ленина, д.104',
   'ИННЮЛ': '1635011279',
   'НаимОрг': 'Общество с ограниченной ответственностью "УНЫШ"'},
  'ОКПО': '47085707',
  'ОКФС': '16',
  'ОКОПФ': '12300',
  'ОКВЭД2': '01.50'},
 'Баланс': {'ОКУД': '0710001',
  'Актив': {'ОбА': {'ДебЗад': {'СумОтч': 62430, 'СумПрдщ': 2678},
    'Запасы': {'СумОтч': 111071, 'СумПрдщ': 100760, 'СумПрдшв': 86693},
    'СумОтч': 173703,
    'ПрочОбА': {},
    'СумПрдщ': 103963,
    'ФинВлож': {},
    'ДенежнСр': {'СумОтч': 202, 'СумПрдщ': 525},
    'СумПрдшв': 86693,
    'НДСПриобрЦен': {}},
   'ВнеОбА': {'ОснСр': {'СумОтч': 60449, 'СумПрдщ': 34200, 'СумПрдшв': 33757},
    'СумОтч': 82252,
    'СумПрдщ': 49487,
    'ФинВлож': {},
    'ВлМатЦен': {},
    'НематАкт': {},
    'СумПрдшв': 45095,
    'ОтлНалАкт': {},
    'РезИсслед': {},
    'ПрочВнеОбА': {'СумОтч': 21803, 'СумПрдщ': 15287, 'СумПрдшв': 11338}},
  

In [65]:
def unpack_lvl_1(data: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Распаковка указанного столбца на один уровень
    :param data: датасет
    :param column_name: название столбца для распаковки
    :return: исходный датасет с распакованными признаками
    """

    if column_name not in data.columns:
        raise ValueError(f"Столбец '{column_name}' не найден в DataFrame")

    unpacked_data = data[column_name].apply(pd.Series)
    data = data.drop([column_name], axis=1)
    data = pd.concat([data, unpacked_data], axis=1)

    return data

In [66]:
df = unpack_lvl_1(df, column_name='balance_data')

Теперь в качестве признаков имеем разделы бухгалтерского отчета:

In [67]:
df

,Inn,god,lik_m,КНД,ОКЕИ,СвНП,Баланс,Период,ФинРез,ЦелИсп,ДатаДок,НомКорр,ПрАудит,СвАудит,ВерсФорм,ОтчетГод,Подписант,ДвижениеДен,ОтчетИзмКап
0,1635011279,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '163501001', 'АдрМН': '422116...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 287558...","{'ОКУД': '0710003', 'Поступило': {'Прочие': {}...",30.03.2023,0,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'Айдар', 'Фамилия': 'Мустафин'...","{'ОКУД': '0710005', 'ИнвОпер': {'Платеж': {'Пр...","{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
1,1628007031,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '162801001', 'АдрМН': '423722...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 862}, ...","{'ОКУД': '0710003', 'Поступило': {'Прочие': {}...",25.03.2023,0,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'Рустам', 'Фамилия': 'Габдрахм...","{'ОКУД': '0710005', 'ИнвОпер': {'Платеж': {'Пр...","{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
2,1632007852,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '163201001', 'АдрМН': '643,42...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 237783...","{'ОКУД': '', 'Поступило': {'Прочие': {}, 'Приб...",27.03.2023,0,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'ФАНИС', 'Фамилия': 'ВАЛЕЕВ', ...","{'ОКУД': '', 'ИнвОпер': {'Платеж': {'ПроцДолгО...","{'ОКУД': '', 'Коррект': {'КапитВс': {'ДоКоррек..."
3,1639049890,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '163901001', 'АдрМН': '423885...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 172142...","{'ОКУД': '', 'Поступило': {'Прочие': {}, 'Приб...",31.03.2023,0,True,"{'АудитОрг': {'ОГРН': '1191690084630', 'ИННЮЛ'...",5.08,2022,"{'ФИО': {'Имя': 'Александр', 'Фамилия': 'Кирши...","{'ОКУД': '0710005', 'ИнвОпер': {'Платеж': {'Су...","{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
4,1677000712,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '167301001', 'АдрМН': '422470...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",94,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 375597...","{'ОКУД': '0710003', 'Поступило': {'Прочие': {}...",02.05.2023,3,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'Иван', 'Фамилия': 'Шугаев', '...","{'ОКУД': '0710005', 'ИнвОпер': {'Платеж': {'Су...","{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26125,7801310906,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '780101001', 'АдрМН': '199178...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 57532}...","{'ОКУД': '', 'Поступило': {'Прочие': {}, 'Приб...",28.03.2023,0,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'Николай', 'Фамилия': 'Бурчула...","{'ОКУД': '', 'ИнвОпер': {'Платеж': {'ПроцДолгО...","{'ОКУД': '', 'Коррект': {'КапитВс': {'ДоКоррек..."
26126,7728461730,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '772801001', 'АдрМН': '117246...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 123899...","{'ОКУД': '', 'Поступило': {'Прочие': {}, 'Приб...",06.02.2023,0,False,"{'АудитОрг': {'ОГРН': '', 'ИННЮЛ': '', 'НаимОр...",5.08,2022,"{'ФИО': {'Имя': 'Станислав', 'Фамилия': 'Колуп...","{'ОКУД': '', 'ИнвОпер': {'Платеж': {'ПроцДолгО...","{'ОКУД': '', 'Коррект': {'КапитВс': {'ДоКоррек..."
26127,1644036086,2022,2099-12-31,0710099,384,"{'НПЮЛ': {'КПП': '164401001', 'АдрМН': 'Респ. ...","{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 754470...","{'ОКУД': '0710003', 'Поступило': {'Прочие': {}...",30.03.2023,0,True,"{'АудитОрг': {'ОГРН': '1051641051780', 'ИННЮЛ'...",5.08,2022,"{'ФИО': {'Им

Распаковка на 1 уровень нужна, чтобы сразу отсеять разделы, с которыми мы не будем работать. 
Оставим основные разделы:
- Баланс - бухгалтерский баланс
- ФинРез - финансовые результаты
- ОтчетИзмКап - отчет об изменениях капитала

In [68]:
def drop_columns(data: pd.DataFrame, columns_to_drop: list) -> pd.DataFrame:
    """
    Удаление столбцов из DataFrame, если они существуют. 
    :param data: датасет
    :param columns_to_drop: список столбцов для удаления
    :return: датасет 
    """

    missing_columns = [
        col for col in columns_to_drop if col not in data.columns
    ]
    if missing_columns:
        print(
            f"Следующих столбцов не существует в датасете: {missing_columns}")
    return data.drop(
        columns=[col for col in columns_to_drop if col in data.columns],
        errors='ignore')

In [69]:
df = drop_columns(df, preproc['drop_sections'])

In [70]:
df[:5]

,Inn,god,lik_m,КНД,ОКЕИ,Баланс,Период,ФинРез,ДатаДок,НомКорр,ПрАудит,ВерсФорм,ОтчетГод,ОтчетИзмКап
0,1635011279,2022,2099-12-31,0710099,384,"{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 287558...",30.03.2023,0,False,5.08,2022,"{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
1,1628007031,2022,2099-12-31,0710099,384,"{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 862}, ...",25.03.2023,0,False,5.08,2022,"{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
2,1632007852,2022,2099-12-31,0710099,384,"{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 237783...",27.03.2023,0,False,5.08,2022,"{'ОКУД': '', 'Коррект': {'КапитВс': {'ДоКоррек..."
3,1639049890,2022,2099-12-31,0710099,384,"{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",34,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 172142...",31.03.2023,0,True,5.08,2022,"{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."
4,1677000712,2022,2099-12-31,0710099,384,"{'ОКУД': '0710001', 'Актив': {'ОбА': {'ДебЗад'...",94,"{'ОКУД': '0710002', 'Выруч': {'СумОтч': 375597...",02.05.2023,3,False,5.08,2022,"{'ОКУД': '0710004', 'Коррект': {'КапитВс': {'Д..."


In [71]:
def unpack_columns(data: pd.DataFrame) -> pd.DataFrame:
    """
    Распаковка многоуровневых признаков
    :param data: датасет
    :return: датасет
    """
    for col in data.columns:
        if isinstance(data[col].iloc[0], dict):
            unpacked = pd.json_normalize(data[col])
            data.drop(columns=[col], inplace=True)
            data = pd.concat([data, unpacked], axis=1)
    return data

In [72]:
df = unpack_columns(df)
df

,Inn,god,lik_m,КНД,ОКЕИ,Период,ДатаДок,НомКорр,ПрАудит,ВерсФорм,...,ДвиженКап.ПредГод.УвеличКапитал.Реорганизация.СобВыкупАкц,ДвиженКап.ПредГод.УменКапитал.Реорганизация.СобВыкупАкц,ДвиженКап.ОтчетГод.УвеличКапитал.УвеличНомАкц.НераспПриб,ДвиженКап.ПредГод.УменКапитал.Реорганизация.РезКапитал,ДвиженКап.ОтчетГод.ИзмДобавКап.РезКапитал,ДвиженКап.ОтчетГод.УвеличКапитал.УвеличНомАкц.СобВыкупАкц,ДвиженКап.ПредГод.УменКапитал.УменНомАкц.ДобКапитал,ДвиженКап.ОтчетГод.УменКапитал.УменНомАкц.ДобКапитал,ДвиженКап.ОтчетГод.УменКапитал.Реорганизация.РезКапитал,ДвиженКап.ОтчетГод.УвеличКапитал.Реорганизация.СобВыкупАкц
0,1635011279,2022,2099-12-31,0710099,384,34,30.03.2023,0,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1628007031,2022,2099-12-31,0710099,384,34,25.03.2023,0,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1632007852,2022,2099-12-31,0710099,384,34,27.03.2023,0,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1639049890,2022,2099-12-31,0710099,384,34,31.03.2023,0,True,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1677000712,2022,2099-12-31,0710099,384,94,02.05.2023,3,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26125,7801310906,2022,2099-12-31,0710099,384,34,28.03.2023,0,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26126,7728461730,2022,2099-12-31,0710099,384,34,06.02.2023,0,False,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26127,1644036086,2022,2099-12-31,0710099,384,34,30.03.2023,0,True,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26128,2502027296,2022,2099-12-31,0710099,384,34,28.03.2023,0,True,5.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Удаление и редактирование признаков

Удалим столбцы с большим кол-вом пропусков:

In [73]:
def remove_columns_with_gaps(data: pd.DataFrame,
                             threshold: float) -> Union[pd.DataFrame, list]:
    """
    Удаление столбцов, где кол-во пропусков превышает заданный порог
    :param data: датасет
    :param threshold: пороговая доля пропусков
    :return: датасет
    :return: список удаленных столбцов
    """
    total_rows = data.shape[0]
    missing_threshold = total_rows * threshold
    columns_to_drop = data.columns[
        data.isnull().sum() > missing_threshold].tolist()

    data.drop(columns=columns_to_drop, inplace=True)

    return data, columns_to_drop

In [74]:
df, drops = remove_columns_with_gaps(df, preproc['threshold'])

In [75]:
preproc['threshold']

0.35

In [76]:
print(f"Удалено столбцов: {len(drops)}")
print(f"Удаленные столбцы: {drops}")

Удалено столбцов: 397
Удаленные столбцы: ['Актив.ВнеОбА.ПрочВнеОбА.СумОтч', 'Актив.ВнеОбА.ПрочВнеОбА.СумПрдщ', 'Актив.ВнеОбА.ПрочВнеОбА.СумПрдшв', 'Пассив.ДолгосрОбяз.ЗаемСредств.СумОтч', 'Пассив.КраткосрОбяз.ЗаемСредств.СумОтч', 'Актив.ОбА.НДСПриобрЦен.СумОтч', 'Актив.ОбА.НДСПриобрЦен.СумПрдщ', 'Пассив.КапРез.ДобКапитал.СумОтч', 'Пассив.КапРез.ДобКапитал.СумПрдщ', 'Пассив.КапРез.ДобКапитал.СумПрдшв', 'Пассив.КапРез.ПереоцВнеОбА.СумОтч', 'Пассив.КапРез.ПереоцВнеОбА.СумПрдщ', 'Пассив.КапРез.ПереоцВнеОбА.СумПрдшв', 'Пассив.ДолгосрОбяз.ПрочОбяз.СумОтч', 'Пассив.ДолгосрОбяз.ЗаемСредств.СумПрдщ', 'Пассив.ДолгосрОбяз.ЗаемСредств.СумПрдшв', 'Пассив.КраткосрОбяз.ПрочОбяз.СумПрдщ', 'Пассив.КраткосрОбяз.ЗаемСредств.СумПрдщ', 'Актив.ОбА.ФинВлож.СумОтч', 'Актив.ОбА.ФинВлож.СумПрдщ', 'Актив.ОбА.НДСПриобрЦен.СумПрдшв', 'Пассив.ДолгосрОбяз.ПрочОбяз.СумПрдщ', 'Пассив.КраткосрОбяз.ОценОбяз.СумОтч', 'Пассив.КраткосрОбяз.ОценОбяз.СумПрдщ', 'Пассив.КраткосрОбяз.ОценОбяз.СумПрдшв', 'Пассив.КраткосрОбяз.Зае

In [77]:
df.shape

(26130, 105)

Некоторые из оставшихся признаков также можно удалить (по ключевому слову):

In [78]:
def remove_columns_with_keyword(data: pd.DataFrame,
                                list_keywords: [list, str]) -> pd.DataFrame:
    """
    Удаление столбцов, содержащих в названии ключевое слово
    :param data: датасет
    :param keyword: список ключевых слов
    :return: датасет
    """
    if isinstance(list_keywords, str):
        list_keywords = [list_keywords]
    columns_to_drop = [
        col for col in data.columns
        if any(keyword.lower() in col.lower() for keyword in list_keywords)
    ]
    data.drop(columns=columns_to_drop, inplace=True)
    return data

In [79]:
df = remove_columns_with_keyword(df, preproc['drop_by_keyword'][0])

In [80]:
preproc['drop_by_keyword'][0]

'коррект.'

In [81]:
df.shape

(26130, 93)

Проанализируем некоторые оставшиеся признаки, удалим лишнее:

In [82]:
df['ВерсФорм'].value_counts()

ВерсФорм
5.08    24456
5.03     1674
Name: count, dtype: int64

In [83]:
def missing_values(data: pd.DataFrame):
    """
    Подсчет и вывод пропусков в столбцах
    :param data: датасет
    """
    columns_with_mv = data.columns[data.isnull().any()]
    for column in columns_with_mv:
        print(
            f'{column}: {round(data[column].isna().sum() / data.shape[0]*100, 1)}% пропусков'
        )

In [84]:
# Посмотрим сколько плохо заполненных признаков содержит версия 5.03 
# Не всегда она есть в данных
filtr = df[df['ВерсФорм'] == '5.03']
missing_values(filtr)

ПрАудит: 100.0% пропусков
Актив.ОбА.ДебЗад.СумОтч: 100.0% пропусков
Актив.ОбА.ДебЗад.СумПрдщ: 100.0% пропусков
Актив.ОбА.Запасы.СумОтч: 100.0% пропусков
Актив.ОбА.Запасы.СумПрдщ: 100.0% пропусков
Актив.ОбА.Запасы.СумПрдшв: 100.0% пропусков
Актив.ОбА.СумОтч: 100.0% пропусков
Актив.ОбА.СумПрдщ: 100.0% пропусков
Актив.ОбА.ДенежнСр.СумОтч: 100.0% пропусков
Актив.ОбА.ДенежнСр.СумПрдщ: 100.0% пропусков
Актив.ОбА.СумПрдшв: 100.0% пропусков
Актив.ВнеОбА.ОснСр.СумОтч: 100.0% пропусков
Актив.ВнеОбА.ОснСр.СумПрдщ: 100.0% пропусков
Актив.ВнеОбА.ОснСр.СумПрдшв: 100.0% пропусков
Актив.ВнеОбА.СумОтч: 100.0% пропусков
Актив.ВнеОбА.СумПрдщ: 100.0% пропусков
Актив.ВнеОбА.СумПрдшв: 100.0% пропусков
Пассив.КапРез.НераспПриб.СумОтч: 100.0% пропусков
Пассив.КапРез.НераспПриб.СумПрдщ: 100.0% пропусков
Пассив.КапРез.НераспПриб.СумПрдшв: 100.0% пропусков
Пассив.КапРез.УставКапитал.СумОтч: 100.0% пропусков
Пассив.КапРез.УставКапитал.СумПрдщ: 100.0% пропусков
Пассив.КапРез.УставКапитал.СумПрдшв: 100.0% пропусков

У компаний сдающих отчет по версии 5.03 почти все признаки не заполнены - эти компании можно удалить

In [85]:
df = df.drop(df[df['ВерсФорм'] == '5.03'].index)

In [86]:
df['Период'].value_counts()

Период
34    24275
94      181
Name: count, dtype: int64

In [87]:
df['god'].value_counts()

god
2022    24456
Name: count, dtype: int64

In [88]:
df['ОтчетГод'].value_counts()

ОтчетГод
2022    24456
Name: count, dtype: int64

In [89]:
df = drop_columns(df, preproc['drop_columns'])

In [90]:
preproc['drop_columns']

['Inn',
 'КНД',
 'ОКЕИ',
 'Период',
 'ОтчетГод',
 'god',
 'НомКорр',
 'ДатаДок',
 'ВерсФорм',
 'ОКУД']

In [91]:
def replace_values(data: pd.DataFrame, change_columns: dict) -> pd.DataFrame:
    """
    Замена значений в датасете
    :param data: датасет
    :param change_columns: словарь со старыми и новыми значениями
    :return: датасет
    """
    for old_str, new_str in change_columns.items():
        data.columns = data.columns.str.replace(old_str, new_str)
    return data

In [92]:
replace_values(data=df, change_columns=preproc['rename_columns'])

,target,ПрАудит,Актив_ОбА_ДебЗад_Отч,Актив_ОбА_ДебЗад_Пред,Актив_ОбА_Запасы_Отч,Актив_ОбА_Запасы_Пред,Актив_ОбА_Запасы_ПредПред,Актив_ОбА_Отч,Актив_ОбА_Пред,Актив_ОбА_ДенежнСр_Отч,...,ЧистАктив_Пред,ЧистАктив_ПредПред,ДвиженКап_ОтчетГод_УвеличКапитал_ЧистПриб_Итог,ДвиженКап_ОтчетГод_УвеличКапитал_ЧистПриб_НераспПриб,ДвиженКап_ОтчетГод_УвеличКапитал_УвеличКапВс_Итог,ДвиженКап_ОтчетГод_УвеличКапитал_УвеличКапВс_НераспПриб,ДвиженКап_ПредГод_УвеличКапитал_ЧистПриб_Итог,ДвиженКап_ПредГод_УвеличКапитал_ЧистПриб_НераспПриб,ДвиженКап_ПредГод_УвеличКапитал_УвеличКапВс_Итог,ДвиженКап_ПредГод_УвеличКапитал_УвеличКапВс_НераспПриб
0,2099-12-31,False,62430.0,2678.0,111071.0,100760.0,86693.0,173703.0,103963.0,202.0,...,0.0,59276.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2099-12-31,False,NaN,NaN,711.0,NaN,NaN,749.0,12.0,NaN,...,9838.0,9838.0,73.0,73.0,73.0,73.0,NaN,NaN,NaN,NaN
2,2099-12-31,False,2141.0,4137.0,131120.0,145374.0,110450.0,154797.0,151635.0,21536.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2099-12-31,True,822600.0,501767.0,189445.0,608825.0,233520.0,6520926.0,1228569.0,510777.0,...,172369.0,135653.0,1864835.0,1864835.0,1864835.0,1864835.0,36716.0,36716.0,36716.0,36716.0
4,2099-12-31,False,135717.0,NaN,272138.0,NaN,NaN,459985.0,NaN,52130.0,...,0.0,0.0,227.0,227.0,237.0,227.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26124,2099-12-31,True,112568.0,101328.0,3.0,353.0,158.0,126227.0,171801.0,13656.0,...,112987.0,137358.0,215903.0,215903.0,215903.0,215903.0,208494.0,208494.0,208494.0,208494.0
26125,2099-12-31,False,4928.0,6632.0,595.0,692.0,685.0,14451.0,13735.0,4326.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26126,2099-12-31,False,43712.0,198362.0,NaN,NaN,NaN,233095.0,322900.0,9886.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26127,2099-12-31,True,98266.0,143515.0,6415.0,6493.0,7562.0,171270.0,154919.0,66589.0,...,39463.0,62010.0,29766.0,29766.0,29787.0,29787.0,NaN,NaN,NaN,NaN


Удалим по ключевому слову признаки, связанные с движением капитала (кроме столбцов с общими итогами, их мы переименовали ранее):

In [93]:
df = remove_columns_with_keyword(df, preproc['drop_by_keyword'][1])

In [94]:
df.shape

(24456, 64)

In [95]:
missing_values(df)

Актив_ОбА_ДебЗад_Отч: 1.7% пропусков
Актив_ОбА_ДебЗад_Пред: 2.0% пропусков
Актив_ОбА_Запасы_Отч: 9.1% пропусков
Актив_ОбА_Запасы_Пред: 7.9% пропусков
Актив_ОбА_Запасы_ПредПред: 7.4% пропусков
Актив_ОбА_Отч: 0.8% пропусков
Актив_ОбА_Пред: 1.4% пропусков
Актив_ОбА_ДенежнСр_Отч: 3.6% пропусков
Актив_ОбА_ДенежнСр_Пред: 3.8% пропусков
Актив_ОбА_ПредПред: 2.1% пропусков
Актив_ВнеОбА_ОснСр_Отч: 12.0% пропусков
Актив_ВнеОбА_ОснСр_Пред: 11.0% пропусков
Актив_ВнеОбА_ОснСр_ПредПред: 10.9% пропусков
Актив_ВнеОбА_Отч: 7.3% пропусков
Актив_ВнеОбА_Пред: 6.9% пропусков
Актив_ВнеОбА_ПредПред: 7.2% пропусков
Актив_Отч: 0.7% пропусков
Актив_Пред: 1.4% пропусков
Актив_ПредПред: 2.1% пропусков
Пассив_КапРез_Отч: 0.7% пропусков
Пассив_КапРез_Пред: 1.5% пропусков
Пассив_КапРез_ПредПред: 2.3% пропусков
Пассив_КапРез_НераспПриб_Отч: 1.1% пропусков
Пассив_КапРез_НераспПриб_Пред: 1.9% пропусков
Пассив_КапРез_НераспПриб_ПредПред: 2.6% пропусков
Пассив_КапРез_УставКапитал_Отч: 1.0% пропусков
Пассив_КапРез_УставКап

Заполним оставшиеся пропуски нулями. Будем исходить из того, что у компаний, оставляющих данные столбцы пустыми соответсвующие показатели на текущий год отсутствуют (требований от ФНС самостоятельно заполнять нулями отсутствующие статьи - нет).

In [96]:
df.fillna(0, inplace=True)

Удалим компании, которые не заполняют активы/пассивы (если не заполнено что-то одно, то не заполнено и второе):

In [97]:
df[df['Актив_Отч'] == 0].shape

(168, 64)

In [98]:
df[df['Актив_Пред'] == 0].shape

(331, 64)

In [99]:
df[df['Актив_ПредПред'] == 0].shape

(505, 64)

In [100]:
df = df.drop(df[(df[preproc['cols_with_null']] == 0).any(axis=1)].index)

In [101]:
df.shape

(23863, 64)

Бинаризуем некоторые признаки:

In [102]:
df['ПрАудит'] = [1 if i==True else 0 for i in df['ПрАудит']]

In [103]:
if 'target' in df.columns:
    df['target'] = [0 if i=='2099-12-31' else 1 for i in df['target']]
else:
    print("Признака 'target' нет в DataFrame")

In [104]:
df

,target,ПрАудит,Актив_ОбА_ДебЗад_Отч,Актив_ОбА_ДебЗад_Пред,Актив_ОбА_Запасы_Отч,Актив_ОбА_Запасы_Пред,Актив_ОбА_Запасы_ПредПред,Актив_ОбА_Отч,Актив_ОбА_Пред,Актив_ОбА_ДенежнСр_Отч,...,ПрибУбДоНал_Отч,ВаловаяПрибыль_Отч,НалПриб_Отч,ТекНалПриб_Отч,ДвижКап_Итог_Пред,ДвижКап_Итог_Отч,ДвижКап_Итог_ПредПред,ЧистАктив_Отч,ЧистАктив_Пред,ЧистАктив_ПредПред
0,0,0,62430.0,2678.0,111071.0,100760.0,86693.0,173703.0,103963.0,202.0,...,58060.0,54521.0,0.0,0.0,47820.0,47820.0,47820,0.0,0.0,59276.0
1,0,0,0.0,0.0,711.0,0.0,0.0,749.0,12.0,0.0,...,94.0,94.0,-19.0,0.0,9838.0,9911.0,9838,9911.0,9838.0,9838.0
2,0,0,2141.0,4137.0,131120.0,145374.0,110450.0,154797.0,151635.0,21536.0,...,2309.0,2309.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,0,1,822600.0,501767.0,189445.0,608825.0,233520.0,6520926.0,1228569.0,510777.0,...,2332819.0,2719107.0,-467984.0,467984.0,172369.0,2037204.0,135653,2037204.0,172369.0,135653.0
6,0,1,147864.0,186224.0,269591.0,299869.0,467362.0,477547.0,555396.0,3884.0,...,5530.0,61166.0,-1631.0,129.0,17569.0,21451.0,15688,21451.0,17569.0,15688.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26124,0,1,112568.0,101328.0,3.0,353.0,158.0,126227.0,171801.0,13656.0,...,260291.0,289304.0,-44388.0,44642.0,112987.0,86407.0,137358,86407.0,112987.0,137358.0
26125,0,0,4928.0,6632.0,595.0,692.0,685.0,14451.0,13735.0,4326.0,...,3079.0,47154.0,-1780.0,0.0,0.0,0.0,0,0.0,0.0,0.0
26126,0,0,43712.0,198362.0,0.0,0.0,0.0,233095.0,322900.0,9886.0,...,61.0,8978.0,-12.0,12.0,0.0,0.0,0,0.0,0.0,0.0
26127,0,1,98266.0,143515.0,6415.0,6493.0,7562.0,171270.0,154919.0,66589.0,...,47285.0,615916.0,-17519.0,16734.0,39463.0,69143.0,62010,69143.0,39463.0,62010.0


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23863 entries, 0 to 26128
Data columns (total 64 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   target                                     23863 non-null  int64  
 1   ПрАудит                                    23863 non-null  int64  
 2   Актив_ОбА_ДебЗад_Отч                       23863 non-null  float64
 3   Актив_ОбА_ДебЗад_Пред                      23863 non-null  float64
 4   Актив_ОбА_Запасы_Отч                       23863 non-null  float64
 5   Актив_ОбА_Запасы_Пред                      23863 non-null  float64
 6   Актив_ОбА_Запасы_ПредПред                  23863 non-null  float64
 7   Актив_ОбА_Отч                              23863 non-null  float64
 8   Актив_ОбА_Пред                             23863 non-null  float64
 9   Актив_ОбА_ДенежнСр_Отч                     23863 non-null  float64
 10  Актив_ОбА_ДенежнСр_Пред    

In [106]:
df.to_csv(preproc['train_path_proc'],
          index=False)

Сохраним уникальные значения до остальных преобразований (понадобится для ввода данных вручную):

In [ ]:
def save_unique_train_data(data: pd.DataFrame,
                           unique_values_path: str,
                           target_column: Optional[str] = None) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param drop_columns: список с признаками для удаления
    :param data: датасет
    :param target_column: целевая переменная
    :param unique_values_path: путь до файла со словарем
    :return: None
    """

    if target_column is not None:
        unique_df = data.drop(columns=[target_column], axis=1, errors="ignore")
    else:
        unique_df = data

    dict_unique = {
        key: unique_df[key].unique().tolist()
        for key in unique_df.columns
    }
    with open(unique_values_path, "w") as file:
        json.dump(dict_unique, file)

In [ ]:
save_unique_train_data(
    data=df,
    target_column=preproc["target_column"],
    unique_values_path=preproc["unique_values_before_preproc"],
)